# Explanation of tree modification code

### Sections of this notebook
**How and Why** - explanations of what we used this code for and the cases of modifications made.  
**Overall code flow** - the general steps of the code.  
**File explanations** - output filenames and what they represent.  
**Visualization explanation** - this code produces a modified tree topology with modified names representing strains for computations and visualizations.  The names also undergo important modifications as explained in this section.   
**Code and tests** - The actual workflow and tests for critical functions.  


### How and Why
For strains in the database we generated short amplicon reads of the V4 regions of the 16s rRNA (primers: 515f, 806r). Many of the strains in our collection are identical over this range, leading to a single 16s feature representing many metabolically/physiologically distinct entities. 

For many of the analyses we perform, we need a phylogenetic tree that has an individual tip for each distinct strain; a 1-1 relationship between strains and tips in the tree. To build this, we first construct the set of all subgraphs (as described below). We then take these subgraphs, and modify the phylogenetic tree reconstructed from the 16s features until it has an exact 1-1 correspondence between tips and strains.

For the purposes of this code, imagine a bipartite graph with a strain as one node type and a 16s features as another node type. Edges in this graph connect strain nodes to the 16s features in their genomes (nodes). Starting at a strain  (s) we want to find the subgraph that includes s. In most cases, a strain has a single, unique 16s feature. So the graph just includes s and its 16s feature (e.g. (s, feature_a)). There are three other cases that we must deal with as detailed in the trees below.


Case 1: a strain has multiple 16s features in the V4 region. In other words, the subgraph contains one strain node and several feature nodes. In this example, `strain3` has two 16s features. To create a phylogeny where there is a 1-1 correspondence between tips and strains, we need to remove tree tips C and D, and replace them with a new tip. The distance between this tip and it's parent is set to the average of distance between C and the parent and D and the parent. 

The table of strain to feature correspondence.

strain	16s_feature  
strain1	A  
strain2	B  
strain3	C,D  

Tree string: ((A:0.1, B:0.1)in1:0.2,(C:0.1, D:0.3)in2:0.2)in3:0.3;  
```
                    /-A
          /in1-----|
         |          \-B
-in3-----|
         |          /-C
          \in2-----|
                    \-D
```
The tree string and visualization after we've combined tips C and D into new tip E.

strain	16s_feature    
strain1	A  
strain2	B  
strain3	E  

After filtration.
Tree string: ((A:0.1, B:0.1)in1:0.2,E:0.4)in3:0.3;  
```
                    /-A
          /in1-----|
-in3-----|          \-B
         |
          \-E
```

There is a more complex version of this case where the last common ancestor of the tips to be combined also has descendants that shouldn't be combined. In the follow example, we want to combine cf137, cf138, cf139, and cf140 into a tip called 'NEW'. However, the LCA of these tips (in121) contains anotehr descendant tip that we can't remove.

strain	16s_feature   
strain1	cf137,cf138,cf139,cf140  
strain2	cfXXX  
strain3	cf134  
strain4	cf135  
strain5	cf136  

Tree string: ((cf137:0.00619,((cf138:0.00394,cf139:0.0,cfXXX:0.4)in123:0.0,cf140:0.00388)in122:0.00567)in121:0.04868,(cf135:0.00218,(cf136:0.00325,cf134:0.0007)in125:0.00177)in124:0.03926)in120:0.00239;   
```
                    /-cf137
                   |
                   |                    /-cf138
          /in121---|                   |
         |         |          /in123---|--cf139
         |         |         |         |
         |          \in122---|          \-cfXXX
-in120---|                   |
         |                    \-cf140
         |
         |          /-cf135
          \in124---|
                   |          /-cf136
                    \in125---|
                              \-cf134
```
After filtration. 
Tree string: ((cfXXX:0.400567, NEW:0.007755)in121:0.04868,(cf135:0.00218,(cf136:0.00325,cf134:0.0007)in125:0.00177)in124:0.03926)in120:0.00239;
```
                    /-cfXXX
          /in121---|
         |          \-NEW
-in120---|
         |          /-cf135
          \in124---|
                   |          /-cf136
                    \in125---|
                              \-cf134
``` 

Case 2: multiple strains share the same 16s feature. In this subgraph, there is one 16s node and multiple strain nodes connected to it. In this example, `strain1` through `strain4` share the same 16s feature. Here, we convert the existing tip to an internal node by adding four 0-length tips. These allow us to have a 1-1 correspondence between strains and tips without introducing any inaccuracies in branchlength. 

The table of strain to feature correspondence.  
strain	16s_feature  
strain1	A    
strain2	A  
strain3	A  
strain4	A  
strain5 B  
strain6	C	  
strain7	D  

The tree string in Newick format and a visualization (without branch length).  
((A:0.1, B:0.1)in1:0.2,(C:0.1, D:0.3)in2:0.2)in3:0.3;
```
                    /-A
          /in1-----|
         |          \-B
-in3-----|
         |          /-C
          \in2-----|
                    \-D
```
We n-furcate tip A into four 0-length tips.   
(((A1:0, A2:0, A3:0, A4:0)A:0.1, B:0.1)in1:0.2,(C:0.1, D:0.3)in2:0.2)in3:0.3;  
The resulting tree looks like the following (again with branchlengths not accurately represented).  
```
                              /-A1
                             |
                             |--A2
                    /A-------|
                   |         |--A3
          /in1-----|         |
         |         |          \-A4
         |         |
-in3-----|          \-B
         |
         |          /-C
          \in2-----|
                    \-D

```
Case 3: All of the remaining cases can be thought of in the same way. The subgraph contains some number of strains and 16s feature nodes in any of a number of relationships. In this example (which is common), we have three related strains. `strain1` has a single feature (A). `strain2` shares feature A with `strain1` and has another feature in its genome (B). `strain3` shares no features with `strain1` but shares a features with `strain2` and thus a path to `strain1`. To resolve situations like these, we remove all tips (16s features) found in this subgraph. We find their last-common-ancestor internal node and find the average length from the deleted tips to that LCA node. We add a new branch from that internal node (of the just calculated length) and then new 0-length tips (as many as there are strains in the subgraph). This introduces some topological and distance inaccuracy, but we felt was the best way to capture the complex relationship between single 16s features and the strains they represent. 

The table of strain to feature correspondence.
strain	16s_feature  
strain1	A  
strain2	A,B  
srrain3	A,D  
strain4	D  
strain5 C  
	

The tree string in Newick format and a visualization  
(((A:0.1, B:0.1)in1:0.1, C:0.1)in3:0.1, D:0.4)in4:0.1;

```
                              /-A
                    /in1-----|
          /in3-----|          \-B
         |         |
-in4-----|          \-C
         |
          \-D
```

Now, we must remove features A, B, and D, and build a new tree. Note that we leave C unchanged because it's not part of the subgraph of the A,B,D features. Also note the length from   
((n1:0, n2:0, n3:0, n4:0)inX:0.25, C:0.1)in4:0.1;

```
                    /-n1
                   |
                   |--n2
          /inX-----|
         |         |--n3
-in4-----|         |
         |          \-n4
         |
          \-C
```
Note that you can easily verify these examples using scikit-bio.
```python
from skbio import TreeNode
import io
f = io.StringIO("((n1:0, n2:0, n3:0, n4:0)in1:0.25, C:0.1)in4:0.1;A")
tre = TreeNode.read(f)
print(tre.ascii_art())
```

### Overall code flow
1. Build subgraphs of strain/features.
2. Modify tree so that toplogy reflects 1-1 relationships using the strain/feature subgraphs.
3. Shear the tree so that strains/features that we don't want to show up are removed.
4. Create visualizations and files necessary to make iToll compatible.

### File explanations
**`input_tree.tre`** - The phylogenetic tree that will be modified. Must have branch lengths and internal nodes named.  
**`modified_tree.tre`** - This is the tree where all topological changes have been made. Node names have also been updated to reflect strain names. Some nodes are named with modifications of strain names to aid visualization as explained in that section of the code (nodes named with `nfurTip` or `cTip`).
**`modified_tree.sheared.tre`** - This tree has all exposed internal nodes removed from `modified_tree.tre`.  
**`modified_tree.sheared.unwanted_removed.tre`** - This tree is the same as above, but with additional tips removed as specified in the code (e.g. to produce a Bacteroidetes) only tree.  
**`strain_names.modified_tree.sheared.unwanted_removed.tre`** - This tree is the `modified_tree.sheared.tre` but with all tips named as strains. In other words, the `nfur` and `cTip` has been removed from the names.  
**`itol_order.txt`** - The order (top to bottom) in which strain's will appear on a dendrogram iTOL tree view from `strain_names.modified_tree.sheared.unwanted_removed.tre`.

### Visualization explanations
If using iTOL, you will likely want to visualize **`modified_tree.sheared.unwanted_removed.tre`** to see where you should add clade markers (e.g. over 0-length tips). You can then overlay the names as shown in **`strain_names.modified_tree.sheared.unwanted_removed.tre`**.  
**Note**: we are introducing some strange tip names like `nfurTip__Desulfovibrio piger ATCC 29098 M2` and `cTip__in45__Bacteroides vulgatus ATCC 8482`. We are introducing `nfurTip` into a strain name every time we have a single 16s feature that maps to multiple strains. We are introducing `cTip` into a every time we combine a set of 16s features because they all represent a subgraph. The reason for doing this is so that we can visualize the changes on an iTOL tree. These names appear in `modified_tree.sheared.unwanted_removed.tre` but not in ``strain_names.modified_tree.sheared.unwanted_removed.tre``.
**Note 2**: iTOL orders the nodes of a tree in an unusual way (in the normal dendrogram layout). To have a preorder traversal layout you must select 'Advanced: Leaf Sorting: None'. 

In [ ]:
# Imports
from collections import defaultdict
import numpy as np
import pandas as pd
from skbio import TreeNode
import io
import re

# Code for generating modified tree

The starting point is two dicts mapping strain names to the names of the 16s features representing those strains. The 16s feature names must be the names found in the phylogenetic tree.

```python
taxonomy_to_features = {'Peptostreptococcus sp. CC14N BEI HM-1051': ['cf095', 'cf096'],
                        'Eubacterium rectale ATCC 33656': ['cf143'],
                        ...}
features_to_taxonomy = {'cf136': ['Eubacterium eligens DSMZ 3376'],
                        'cf137': ['Anaerobutyricum hallii DSM 3353'],
                        'cf138': ['Anaerobutyricum hallii DSM 3353'],
                        ...}
```

Note that if you wanted to modify the tree for only a subset of the tips you could reduce these two dicts to just the strains/tips of interst.

In [ ]:
# Ingest culture and strain data to produce required dicts. This code is required because of
# the specific formatting of our strain and culture database. You should bypass/modify
# depending on the way you store this information.
strains_and_cultures_db = '../supplemental_table_6.xlsx'
sac = pd.read_excel(strains_and_cultures_db, index_col=0, sheet_name='cultures')
taxonomies = pd.read_excel(strains_and_cultures_db, index_col=0, sheet_name='full_taxonomy')

# Join taxonomy data with sac to link culture to taxonomic rank information
tmp = ['taxonomy', 'clean_by_16s', '16s_feature', 'kingdom', 'phylum', 'class',
       'order', 'family', 'genus', 'species', 'strain', 'morphology',
       'supplier', 'number']
strain_metadata = sac.join(taxonomies, on='taxonomy')[tmp]

# Build dict's we will use to find subgraph as described above.
taxonomy_to_cf = defaultdict(list)
cf_to_taxonomy = defaultdict(list)

for t, cf in strain_metadata[['taxonomy', '16s_feature']].values:
    if pd.isnull(cf):
        pass
    else:
        taxonomy_to_cf[t].extend(cf.split(','))
        for _cf in cf.split(','):
            cf_to_taxonomy[_cf].append(t)

t2c = {k: sorted(set(v)) for k, v in taxonomy_to_cf.items()}
c2t = {k: sorted(set(v)) for k, v in cf_to_taxonomy.items()}

### Build a new tree (if necessary)
We need to align sequences from our cultures and build a tree. This step can be skipped if you've alread built the tree.

In [ ]:
seqs = pd.read_excel(strains_and_cultures_db, index_col=0, sheet_name='v4_16s')
seq_fasta = []
for cn, seq in zip(seqs.index, seqs['sequence'].values):
    seq_fasta.append('>%s\n%s' % (cn, seq))
out_fp = 'seqs_to_align.fasta'
o = open(out_fp, 'w')
o.writelines('\n'.join(seq_fasta))
o.close()
# Now align with whatever program. Here we'll use CLUSTALW via the MUSCLE pipeline at EBI,

Now you will need to build the tree with these sequences. The next blocks of code assume you've created a tree, which I've called `clustalw_tree.tre`.

### Name internal nodes of the tree
This is required to find nodes and last common ancestors. If a tree already has internal node names, this step is unnecessary.

In [ ]:
tree_fp = 'clustalw_tree.tre'
tr = TreeNode.read(tree_fp, format='newick')

i = 0
for node in tr.preorder():
    if node.name == None:
        node.name = 'in%s' % i
        i += 1
out_fp = 'base_tree.named_inodes.tre'
tr.write(out_fp, format='newick')
tr = TreeNode.read('base_tree.named_inodes.tre', format='newick')

### Build subgraphs

#### Functions

In [ ]:
def strains_from_features(features_16s, feature_to_strain):
    tmp = []
    for f in features_16s:
        tmp.extend(feature_to_strain[f])
    return sorted(set(tmp))

def features_from_strains(strains, strain_to_feature):
    tmp = []
    for s in strains:
        tmp.extend(strain_to_feature[s])
    return sorted(set(tmp))

def find_subgraph(start_features, feature_to_strain, strain_to_feature):
    _16s_features = start_features
    _strains = strains_from_features(start_features, feature_to_strain)
    subgraph = False
    while not subgraph:
        tmp1 = features_from_strains(_strains, strain_to_feature)
        tmp2 = strains_from_features(tmp1, feature_to_strain)
        if (tmp1 == _16s_features) and (tmp2 == _strains):
            subgraph = True
        else:
            subgraph = False
        _16s_features = tmp1
        _strains = tmp2

    return (_16s_features, _strains)

#### Test code
Ensure that we are finding subgraphs like expected.

In [ ]:
feature_to_strain_1 = {'cf01': ['bt1', 'bt2'],
                       'cf02': ['cs1'],
                       'cf03': ['am1', 'am2'],
                       'cf04': ['am2'],
                       'cf05': ['fp1', 'fp2'],
                       'cf06': ['fp1', 'fp2']}

strain_to_feature_1 = {'bt1': ['cf01'],
                       'bt2': ['cf01'],
                       'cs1': ['cf02'],
                       'am1': ['cf03'],
                       'am2': ['cf03', 'cf04'],
                       'fp1': ['cf05', 'cf06'],
                       'fp2': ['cf05', 'cf06']}

obs1, obs2 = find_subgraph(['cf01'], feature_to_strain_1, strain_to_feature_1)
assert obs1 == ['cf01']
assert obs2 == ['bt1', 'bt2']

obs1, obs2 = find_subgraph(['cf02'], feature_to_strain_1, strain_to_feature_1)
assert obs1 == ['cf02']
assert obs2 == ['cs1']

obs1, obs2 = find_subgraph(['cf03'], feature_to_strain_1, strain_to_feature_1)
assert obs1 == ['cf03', 'cf04']
assert obs2 == ['am1', 'am2']

obs1, obs2 = find_subgraph(['cf04'], feature_to_strain_1, strain_to_feature_1)
assert obs1 == ['cf03', 'cf04']
assert obs2 == ['am1', 'am2']

obs1, obs2 = find_subgraph(['cf05'], feature_to_strain_1, strain_to_feature_1)
assert obs1 == ['cf05', 'cf06']
assert obs2 == ['fp1', 'fp2']

#### Create subgraphs

In [ ]:
# Create graphs from bipartite network defined by `t2c` and `c2t`.
remaining_16s_features = sorted(c2t.keys())
subgraphs = []
while len(remaining_16s_features) > 0:
    _16s_features, _strains = find_subgraph([remaining_16s_features[0]], c2t, t2c)
    for i in _16s_features:
        remaining_16s_features.pop(remaining_16s_features.index(i))
    subgraphs.append((_16s_features, _strains))

### Modify the tree using identified subgraphs
#### Define functions to modify a `skbio.TreeNode`

In [ ]:
def prune(tr, tips_to_remove):
    ntr = tr.copy()
    # get lca before removal
    lca_name = ntr.lca(tips_to_remove).name
    # find average distance from lca to tips to be removed
    bl = 0.
    for tip in tips_to_remove:
        bl += ntr.find(tip).accumulate_to_ancestor(ntr.find(lca_name))
    avg_bl = bl / len(tips_to_remove)
    # remove tips
    ntr.remove_deleted(lambda node: node.name in set(tips_to_remove))
    return ntr, lca_name, avg_bl

def prune_and_combine_into_1(tr, tips_to_remove, new_tip_name):
    ntr = tr.copy()
    ntr_1, lca_name, avg_bl = prune(tr, tips_to_remove)
    # In some topologies, after tip removal, the lca node will not be a tip, but will 
    # have descendant internal nodes and tips. We have two cases that differ in how we
    # set branch length.
    if set([tip.name for tip in tr.find(lca_name).tips()]) == set(tips_to_remove):
        lca_bl = ntr_1.find(lca_name).length
        ntr_1.find(lca_name).parent.children.append(TreeNode(name=new_tip_name, length=avg_bl + lca_bl))
        # Delete the previous LCA to remove every internal node that might still be attached.
        ntr_1.remove_deleted(lambda node: node.name == lca_name)
    else:
        # Here we have other tips that descend from the LCA that aren't getting cleaned up. Add new node
        # with appropriate branch length.
        ntr_1.find(lca_name).children.append(TreeNode(name=new_tip_name, length=avg_bl))
        # Prune away the possibly meaningless internal nodes.
        ntr_1.prune()
    return ntr_1

def nfurcate_tip(tr, tip, tips_to_add):
    ntr = tr.copy()
    for tip_name in tips_to_add:
        ntr.find(tip).children.append(TreeNode(name=tip_name, length=0.0))
    return ntr

def add_0_length_tips(tr, lca_name, tips_to_add):
    ntr = tr.copy()
    for tip_name in tips_to_add:
        ntr.find(lca_name).children.append(TreeNode(name=tip_name, length=0.0))
    return ntr

def make_clade(tr, tips_to_remove, tips_to_add, new_inode_name):
    ntr = tr.copy()
    ntr_1, lca_name, avg_bl = prune(ntr, tips_to_remove)
    # Add the new internal node that will be parent to the new tips.
    ntr_1.find(lca_name).children.append(TreeNode(name=new_inode_name, length=avg_bl))
    ntr_2 = add_0_length_tips(ntr_1, new_inode_name, tips_to_add)
    return ntr_2

#### Test code
Ensure our functions modify topology and length as we expect

In [ ]:
tr_str1 = "((A:0.1, B:0.34)in1:0.2,(C:0.1, D:0.3)in2:0.2)in3:0.3;"
tr_str2 = "(((A1:0, A2:0, A3:0, A4:0)A:0.5, B:0.1)in1:0.2,(C:0.1, D:0.3)in2:0.2)in3:0.3;"

tr_str3 = "((C:0.1, D:0.1)B:0.1, (F:0.1, G:0.1)E:0.1)A:0.1;"
exp_tr_str3 = "(B:0.1, (N1:0.0, N2:0.0, N3:0.0)n_inode:0.2, (F:0.1)E:0.1)A:0.1;"

tr_str4 = "(((C:0.1, D:0.1)B:0.1, (F:0.1, G:0.1)E:0.1)A:0.1, (X1:0.1, X2:0.1)X3:0.1)root:0.1;"
exp_tr_str4 = "((B:0.1, (N1:0.0, N2:0.0, N3:0.0)n_inode:0.2, (F:0.1)E:0.1)A:0.1, (X1:0.1, X2:0.1)X3:0.1)root:0.1;"

tr_str5 = "((cf137:0.00619,((cf138:0.00394,cf139:0.0)in123:0.0,cf140:0.00388)in122:0.00567)in121:0.04868,(cf135:0.00218,(cf136:0.00325,cf134:0.0007)in125:0.00177)in124:0.03926)in120:0.00239;"
exp_tr_str5 = "(NEW:0.056435,(cf135:0.00218,(cf136:0.00325,cf134:0.0007)in125:0.00177)in124:0.03926)in120:0.00239;"

tr_str6 = "((cf137:0.00619,((cf138:0.00394,cf139:0.0,cfXXX:0.4)in123:0.0,cf140:0.00388)in122:0.00567)in121:0.04868,(cf135:0.00218,(cf136:0.00325,cf134:0.0007)in125:0.00177)in124:0.03926)in120:0.00239;"
exp_tr_str6 = "((cfXXX:0.400567, NEW:0.007755)in121:0.04868,(cf135:0.00218,(cf136:0.00325,cf134:0.0007)in125:0.00177)in124:0.03926)in120:0.00239;"

# prune
_test_tr = TreeNode.read(io.StringIO(tr_str1))
obs_tr, obs_lca, obs_bl = prune(_test_tr, ['A', 'B'])
assert obs_lca == 'in1'
np.testing.assert_allclose(obs_bl, 0.22)

_test_tr = TreeNode.read(io.StringIO(tr_str2))
obs_tr, obs_lca, obs_bl = prune(_test_tr, ['A1', 'A2'])
assert obs_lca == 'A'
np.testing.assert_allclose(obs_bl, 0.0)

obs_tr, obs_lca, obs_bl = prune(_test_tr, ['A1', 'A2', 'C'])
assert obs_lca == 'in3'
np.testing.assert_allclose(obs_bl, 1.7/3)

# make_clade
_test_tr = TreeNode.read(io.StringIO(tr_str3))
obs_tr = make_clade(_test_tr, ['C', 'D', 'G'], ['N1', 'N2', 'N3'], 'n_inode')
exp_tr = TreeNode.read(io.StringIO(exp_tr_str3))
assert obs_tr.compare_rfd(exp_tr) == 0.0
assert obs_tr.compare_tip_distances(exp_tr) == 0.0

_test_tr = TreeNode.read(io.StringIO(tr_str4))
obs_tr = make_clade(_test_tr, ['C', 'D', 'G'], ['N1', 'N2', 'N3'], 'n_inode')
exp_tr = TreeNode.read(io.StringIO(exp_tr_str4))
assert obs_tr.compare_rfd(exp_tr) == 0.0
assert obs_tr.compare_tip_distances(exp_tr) == 0.0

# prune_and_combine_into_1
_test_tr = TreeNode.read(io.StringIO(tr_str5))
obs_tr = prune_and_combine_into_1(_test_tr, ['cf137', 'cf138', 'cf139', 'cf140'], 'NEW')
exp_tr = TreeNode.read(io.StringIO(exp_tr_str5))
# NEW_bl = 0.04868 + 0.007755
#avg_bl = 0.007755
#(in122 + in123 + cf138) + (in122 + in123 + cf139) + (in122 + cf140) + (cf137)
#(0.00567 + 0 + 0.00394) + (0.00567 + 0 + 0) + (0.00567 + 0.00388) + (0.00619)
assert obs_tr.compare_rfd(exp_tr) == 0.0
assert obs_tr.compare_tip_distances(exp_tr) == 0.0
assert obs_tr.find('NEW').length == 0.056435

_test_tr = TreeNode.read(io.StringIO(tr_str6))
obs_tr = prune_and_combine_into_1(_test_tr, ['cf137', 'cf138', 'cf139', 'cf140'], 'NEW')
exp_tr = TreeNode.read(io.StringIO(exp_tr_str6))
assert obs_tr.compare_rfd(exp_tr) == 0.0
np.testing.assert_allclose(obs_tr.compare_tip_distances(exp_tr), 0.0, rtol=0, atol=1e-5)
np.testing.assert_allclose(obs_tr.find('NEW').length, 0.007755)

#### Modify tree
This code modifies the topology of the tree to resolve the three cases outlined in explanation. It also updates the names of the tips so that they match the supplied strain names. This is required for visualization and because nodes need unique names in an `skbio.TreeNode`.

**Note**: we are introducing some strange tip names like `nfurTip__Desulfovibrio piger ATCC 29098 M2` and `cTip__in45__Bacteroides vulgatus ATCC 8482`. See explanation in **Visualization explanation** in first cell.

In [ ]:
# Load tree
base_tree_fp = 'base_tree.named_inodes.tre'
tr = TreeNode.read(base_tree_fp, format='newick')

# Keep track of the tips we want to leave in the final tree. This tracks the changes we've
# made
tips_to_keep = []
changes = []
# [case, _16s_features, _strains, removed_tipnames, added_tipnames]

# Keep track of the new internal node names that are being used.
new_inode_counter = 0

for _16s_features, _strains in subgraphs:
    n_16s_features = len(_16s_features)
    n_strains = len(_strains)
    
    if (n_16s_features == 1) and (n_strains == 1):
        # No editing needs to be done.
        changes.append((0, _16s_features, _strains, _16s_features, _strains))
        
        #rename.
        tr.find(_16s_features[0]).name = _strains[0]
        tips_to_keep.append(_strains[0])

    elif (n_16s_features > 1) and (n_strains == 1):
        # Case 1
        tr = prune_and_combine_into_1(tr, _16s_features, _strains[0])
        tips_to_keep.append(_strains[0])

        changes.append((1, _16s_features, _strains, _16s_features, _strains[0]))
        
    elif (n_16s_features == 1) and (n_strains > 1):
        # Case 2
        new_names = ['nfurTip__%s' % i for i in _strains]
        tr = nfurcate_tip(tr, _16s_features[0], new_names)
        # Add new tips to keep.
        tips_to_keep.extend(new_names)
        changes.append((2, _16s_features, _strains, _16s_features, new_names))

    else:
        # Case 3
        new_tip_names = ['cTip__%s__%s' % (tr.lca(_16s_features).name, i) for i in 
                         _strains]
        new_inode_name = 'new_inode_' + str(new_inode_counter).zfill(3)
        new_inode_counter += 1
        tr = make_clade(tr, _16s_features, new_tip_names, new_inode_name)

        tips_to_keep.extend(new_tip_names)


        changes.append((2, _16s_features, _strains, _16s_features, new_tip_names))

# Write the resulting file.
out_fp = 'modified_tree.tre'
tr.write(out_fp)

## Make final modifications to tree for visualization
#### Shearing
Because of the topology changes, some internal nodes have been exposed as tips. We need to remove these and shear them back until we reach a node with an LCA tip with a tip we want.

In [ ]:
# Sanity check that all tips we want to keep are in the tree
tips = [i.name for i in tr.tips()]
assert len(set(tips_to_keep).difference(tips)) == 0
# Shear.
out_fp = 'modified_tree.sheared.tre'
sheared_tr = tr.shear(tips_to_keep)
sheared_tr.write(out_fp)

#### Remove and rename tips, and reshear
In some cases, we may only want a subset of the tips (e.g. make a tree for visualization with only the Bacteroidetes). Here we will translate the tip names to those we want and produce twin files with things named only as strains, and those named with e.g. 'cTip' to add our visualization in iTOL.

**Note**: for some reason, certain tips show up with quotation marks upon insertion. I've removed these by hand with simple find and replace in the final files to be visualized.

In [ ]:
# You could substitue any list of tips here. `tips_to_keep` is the set of strain names we want to retain.
data_fp = '../supplemental_table_7.xlsx'
j_agg_md = pd.read_excel(io=data_fp, sheet_name='agggregated_md', index_col=0)
mega_media_samples = ((j_agg_md['sample_type'] == 'supernatant') &
                      (j_agg_md['media'] == 'mm'))
ss_md = j_agg_md.loc[mega_media_samples, :]
tips_to_retain = list(set(ss_md['taxonomy'].values))

# _tmp_tr1 will be the modified tree with unwanted tips removed but with
# the names retained (e.g. there will 'nfurTip' etc in there).
_tmp_tr1 = sheared_tr.copy() 
_tmp_tr1_tips_to_keep = []

# _tmp_tr2 will have the unwanted tips removed, and everything renamed.
_tmp_tr2 = sheared_tr.copy() 

name_map = {}

for tip in sheared_tr.tips():
    existing_name = tip.name
    if 'nfurTip__' in existing_name:
        new_name = existing_name.replace('nfurTip__', '')
    elif 'cTip' in existing_name:
        new_name = re.sub('cTip__in[0-9]+__', '', existing_name)
    else:
        new_name = existing_name
    # Sanity check
    if existing_name in name_map:
        raise ValueError('reused name')
    
    name_map[existing_name] = new_name
    
    if new_name not in tips_to_retain:
        pass
    else:
        _tmp_tr1_tips_to_keep.append(existing_name)
        _tmp_tr2.find(existing_name).name = new_name

# Invalidate caches so name changes take effect
_tmp_tr2.invalidate_caches()
        
unwanted_removed = _tmp_tr1.shear(_tmp_tr1_tips_to_keep)
out_fp = 'modified_tree.sheared.unwanted_removed.tre'
unwanted_removed.write(out_fp)

strain_names_unwanted_removed = _tmp_tr2.shear(tips_to_retain)
out_fp = 'strain_names.modified_tree.sheared.unwanted_removed.tre'
strain_names_unwanted_removed.write(out_fp)

# Write out order of the names
tip_order = []
for node in strain_names_unwanted_removed.preorder():
    if node.is_tip():
        tip_order.append(node.name)

out_fp = 'itol_order.txt'
o = open(out_fp, 'w')
o.writelines('\n'.join(tip_order))
o.close()